In [2]:
import reflexgenerator.sources
from reflexgenerator.sources import (
    Register, BitMask, Bit, IOElement, Metadata, Collection)
from reflexgenerator.sources.io import load

from pathlib import Path
import pandas as pd


In [3]:
schema = r"schema\device.json"
fw_target = r"schema\example.yml"


In [4]:
jsondata = load(fw_target)


In [5]:
metadata = Metadata(
    device=jsondata["device"],
    whoAmI=jsondata["whoAmI"],
    architecture=jsondata["architecture"],
    firmwareVersion=jsondata["firmwareVersion"],
    hardwareVersion=jsondata["hardwareVersion"])
print(metadata)

Metadata(device='ExampleDevice', whoAmI=0, firmwareVersion='0.1', hardwareVersion='0.1', architecture='atmega')


In [6]:
# build masks
masks = Collection([BitMask(**mask) for mask in jsondata["masks"]])
masks_df = pd.DataFrame([mask.to_dict() for mask in masks])
masks_df.fillna("N/A",inplace=True)
masks_df.head(5)

,name,payloadType,description,bits,_uref
0,DO,PayloadType.U8,Bitmask representing the state of the digital ...,"[{'name': 'DO0', 'value': 1}, {'name': 'DO1', ...",Reference to DO


In [7]:
# build registers
regs = Collection([Register(**reg) for reg in jsondata["registers"]])
register_df = pd.DataFrame([reg.to_dict() for reg in regs])
register_df.fillna("N/A",inplace=True)
register_df.head(5)



,name,address,payloadType,alias,arrayType,registerType,maskType,description,converter,visibility,group,_uref
0,Cam0Event,32,PayloadType.U8,N/A,1,RegisterType.Event,N/A,None,N/A,VisibilityType.Public,None,Reference to Cam0Event
1,Cam0TriggerFrequency,33,PayloadType.U16,N/A,1,RegisterType.Command,"{'name': 'DO', 'payloadType': PayloadType.U8, ...",Sets the trigger frequency for camera 0 betwee...,N/A,VisibilityType.Public,None,Reference to Cam0TriggerFrequency
2,Cam0TriggerDuration,34,PayloadType.U16,N/A,1,RegisterType.Command,N/A,Sets the duration of the trigger pulse (minimu...,N/A,VisibilityType.Public,None,Reference to Cam0TriggerDuration
3,StartAndStop,35,PayloadType.U8,N/A,1,RegisterType.Command,N/A,Starts or stops the camera immediately.,N/A,VisibilityType.Public,None,Reference to StartAndStop
4,InState,36,PayloadType.U8,N/A,1,RegisterType.Event,N/A,Contains the state of the input ports.,N/A,VisibilityType.Public,None,Reference to InState


In [8]:
ios = Collection([IOElement(**io) for io in jsondata["ios"]])
ios_df = pd.DataFrame([io.to_dict() for io in ios])
ios_df.fillna("N/A",inplace=True)
ios_df.head(5)

,name,port,pin,direction,useInput,pull,sense,interruptPriority,interruptNumber,out,outDefault,outInvert,description
0,DO3,PORTC,0,output,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,DO0
1,DO2,PORTB,2,output,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,DO0


In [11]:
txt = f"""Registers

{register_df.to_markdown()}


Masks

{masks_df.to_markdown()}


"""

with open("documentation.md", "w") as text_file:
    text_file.write(txt)


In [15]:
from reflexgenerator.generator.markdown import AnchorReference

obj = AnchorReference.REFERENCES["DO"]
obj.parent


from reflexgenerator.generator.markdown import make_anchor, create_link

refname = "Com0Event"
register_df.loc[0,"name"] = make_anchor(refname)

masks_df["TestReference"] = masks_df.apply(lambda  x: create_link(refname, None), axis=1)

register_df["name"] =

BitMask(name='DO', payloadType=<PayloadType.U8: 1>, description='Bitmask representing the state of the digital outputs.', bits=[Bit(name='DO0', value=1), Bit(name='DO1', value=2)], _uref=Reference to DO)